# Ingestion de la carpeta "movie_company"

In [0]:
dbutils.widgets.text("p_environment", "")
v_env = dbutils.widgets.get("p_environment")
v_env

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
movie_companies_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("companyId", IntegerType(), True),
])

In [0]:
movies_companies_df = spark.read.csv(f'{bronze_folder_path}/movie_company/movie_company_*.csv', schema=movie_companies_schema)

In [0]:
movies_companies_df.count()

In [0]:
movies_companies_df.printSchema()

### Paso 2 - Renombrar columnas y adicionar columnas "ingestion_date" y "env"

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
movies_companies_final_df = movies_companies_df \
    .withColumnRenamed("movieId", "movie_id") \
    .withColumnRenamed("companyId", "company_id") \
    .transform(add_ingestion_date) \
    .withColumn('env',lit(v_env))

In [0]:
display(movies_companies_final_df)

### Paso 3 - Escribir datos en el DataLake en formato parquet

In [0]:
# movies_companies_final_df.write.mode('overwrite').parquet(f'{silver_folder_path}/movies_companies')
movies_companies_final_df.write.mode('overwrite').format("parquet").saveAsTable("movie_silver.movies_companies")

In [0]:
%python
display(dbutils.fs.ls("/mnt/moviehistory/silver/movies_companies"))

In [0]:
df = spark.read.parquet('/mnt/moviehistory/silver/movies_companies')
display(df)

In [0]:
dbutils.notebook.exit("Sucess")